##Loading dependencies:

In [ ]:
!pip install pandas numpy fasttext scikit-learn
from sklearn.svm import SVC  # Import the SVC class
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import fasttext as fasttext

##Importing Dataset:

In [ ]:
# Load the datasets
train_data = pd.read_csv("ICONTrain.csv")
test_data = pd.read_csv("ICONTest.csv")


X_train = train_data['sentence'].values
y_train = train_data['sentiment'].values

X_test = test_data['sentence'].values
y_test = test_data['sentiment'].values

In [ ]:
# Save the data in FastText format
with open("train.txt", "w", encoding="utf-8") as f:
    for sentence, tag in zip(X_train, y_train):
        f.write(f"__label__{tag} {sentence}\n")

##Generating fasttext embeddings:

In [ ]:
# Train FastText supervised model
model = fasttext.train_supervised('train.txt', label_prefix='__label__')

def sentence_embedding(sentence, model):
    if not pd.isna(sentence):  # Check for NaN values
        words = sentence.split()
        embeddings = [model[word] for word in words if word in model.words]
        if len(embeddings) == 0:
            return np.zeros(model.get_dimension())
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.get_dimension())  # Return zeros for NaN sentences

# Compute average embeddings for train, test, and eval sets
X_train_avg = [sentence_embedding(text, model) for text in X_train]
X_test_avg = [sentence_embedding(text, model) for text in X_test]


##Training SVM on data:

In [ ]:
# Initialize and train the SVM classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train_avg, y_train)

# Make predictions on test set
y_pred = svm_classifier.predict(X_test_avg)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy}")

Accuracy on test set: 0.7321727485771677
